In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import LabelEncoder
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import regex as re
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import csv
from nltk.stem import PorterStemmer
import string
from nltk.stem import WordNetLemmatizer
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from transformers import RobertaTokenizer, AutoModel,AutoTokenizer,AutoModelForQuestionAnswering

## constant

In [ ]:
train_path     = "/kaggle/input/lmsys-chatbot-arena/train.csv"
test_path      = "/kaggle/input/lmsys-chatbot-arena/test.csv"
model_path     = '../input/transformers/roberta-base'



In [ ]:
train = pd.read_csv(train_path)
test  = pd.read_csv(test_path)

In [ ]:
train

In [ ]:
target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']

In [ ]:
y = train[target_columns].idxmax(axis=1) 

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
labels = torch.tensor(y_encoded, dtype=torch.int64)
labels

In [ ]:
train["all"] = train.apply(lambda train: f"User prompt: {train['prompt']}\n\nModel A :\n{train['response_a']}\n\n--------\n\nModel B:\n{train['response_b']}", axis=1).tolist()


In [ ]:
test["all"] =test.apply(lambda test: f"User prompt: {test['prompt']}\n\nModel A :\n{test['response_a']}\n\n--------\n\nModel B:\n{test['response_b']}", axis=1).tolist()

In [ ]:
X_train=train["all"].tolist()

In [ ]:
X_test=test["all"].tolist()

## Prepare data for Model

### 1-load tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt",max_length=512)

In [ ]:
test_encodings = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt",max_length=512)

In [ ]:
train_encodings

### 2-custom dataset for train

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = CustomDataset(train_encodings, labels)

### 2-custom dataset for test

In [ ]:
class CustomTestDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [ ]:
test_dataset = CustomTestDataset(test_encodings)

### DataLoader


In [ ]:
batch_size = 20
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Model

In [ ]:
num_classes = 3
num_epochs = 8

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(model_path, num_labels=num_classes)



In [ ]:
model = model.to("cuda")
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0,num_training_steps=len(train_loader) * num_epochs)
     

In [ ]:
# Replace with your actual number of epochs

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch_idx, batch in enumerate(train_loader):

        # Move data to GPU
        batch = {key: val.to('cuda') for key, val in batch.items()}
            
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        if (batch_idx + 1) % 300 == 0:  
            print(f"Epoch {epoch + 1}/{num_epochs} - Batch {batch_idx + 1}/{len(train_loader)} - Loss: {total_loss / 100:.4f}")
            total_loss = 0.0
        



In [ ]:
model.eval() 

In [ ]:
all_predictions = []

with torch.no_grad():
    for batch in test_loader:
        # Ensure batch is a dictionary
        if isinstance(batch, dict):
            # Move batch to GPU if available
            batch = {key: val.to('cuda') for key, val in batch.items()}

            # Make predictions
            outputs = model(**batch)
            logits = outputs.logits

            # Apply softmax to get probabilities
            probabilities = torch.softmax(logits, dim=-1)

            # Append probabilities to the list
            all_predictions.append(probabilities.cpu().numpy())
        else:
            print(f"Encountered a non-dict batch: {batch}")

# Concatenate all predictions
all_predictions = np.concatenate(all_predictions, axis=0)

# Prepare DataFrame with predictions
submission = pd.DataFrame(all_predictions, columns=['response_b_len', 'prompt_len', 'winner'])


In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
model_save_path="/kaggle/working/model/"

In [ ]:
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)